In [ ]:
import csv

with open("data/songs.csv") as f:
    reader = csv.reader(f)
    header = next(reader)
    data = [row for row in reader] 

# Part 1

In [ ]:
from transformers import GPT2TokenizerFast, PreTrainedTokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained('openai-community/gpt2')
tokenizer.bos_token = '<s>'
tokenizer.eos_token = '</s>'
tokenizer.pad_token = '<|endoftext|>'

In [ ]:
import numpy as np
from tqdm import tqdm
from typing import List, Union, Dict
from a2_p1_murugan_116745378 import TrigramLM, get_perplexity

In [ ]:
tokenized_data = [tokenizer.tokenize(row[2]) for row in data]
# TODO: check if there is an issue due to sequence length > 1024
# TODO: check if custom newline handling is needed

In [ ]:
lmodel = TrigramLM(tokenizer)
lmodel.train([i[2] for i in data])

In [ ]:
"|<endoftext>|" in tokenizer.vocab

In [ ]:
lmodel.nextProb(["I"], ["Ġremember", "Ġwhen", "ĠI", "Ġwas", "Ġyoung"])

In [ ]:
lmodel.nextProb([], ["<s>"])

In [ ]:
lmodel.nextProb(["I", "Ġremember"], ["Ġwhen", "ĠI", "Ġwas", "Ġyoung"])

In [ ]:
lmodel.nextProb(["<s>", "I"], ["Ġremember", "Ġwhen", "ĠI", "Ġwas", "Ġyoung"])

In [ ]:
prob = lmodel.get_sequence_probability(["I", "Ġremember", "Ġwhen"])

In [ ]:
get_perplexity(prob)

# Part 2

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained('openai-community/gpt2')

print(tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token, tokenizer.unk_token)
print(tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.pad_token_id, tokenizer.unk_token_id)
print(tokenizer.vocab_size)

In [ ]:

tokenizer.bos_token = '<s>'
tokenizer.eos_token = '</s>'
tokenizer.pad_token = '<|endoftext|>'

tokenizer.add_tokens(["<s>", "</s>"])

print(tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token, tokenizer.unk_token)
print(tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.pad_token_id, tokenizer.unk_token_id)
print(tokenizer.vocab_size)

In [ ]:
"<s>" in tokenizer.vocab
tokenizer.vocab["</s>"]

In [ ]:
import torch

token_ids = tokenizer.encode(data[0][2])

chunk_len = 128 - 2
pad_count = (chunk_len - len(token_ids) % chunk_len) if len(token_ids) % chunk_len != 0 else 0

token_ids += [0] * pad_count
chunked_token_ids = torch.tensor(token_ids).reshape(-1, chunk_len)

bos_tensor = torch.full((chunked_token_ids.shape[0], 1), -1)
eos_tensor = torch.full((chunked_token_ids.shape[0], 1), -2)

torch.cat((bos_tensor, chunked_token_ids, eos_tensor), dim=1).shape

chunked_token_ids.shape

In [ ]:

def chunk_tokens(tokens, start_token_id, end_token_id, pad_token_id, chunk_len=128):
    u_chunk_len = chunk_len - 2
    # padding
    pad_count = (u_chunk_len - len(tokens) % u_chunk_len) if len(tokens) % u_chunk_len != 0 else 0
    tokens += [pad_token_id] * pad_count

    # chunking
    chunked_tokens = torch.tensor(tokens).reshape(-1, u_chunk_len)

    # adding start and end tokens
    bos_tensor = torch.full((chunked_tokens.shape[0], 1), start_token_id)
    eos_tensor = torch.full((chunked_tokens.shape[0], 1), end_token_id)
    chunks = torch.cat((bos_tensor, chunked_tokens, eos_tensor), dim=1)

    return chunks

chunk_tokens(
    tokenizer.encode("Hello, how are you?"),
    tokenizer.bos_token_id,
    tokenizer.eos_token_id,
    tokenizer.pad_token_id,
    10
)

In [ ]:
import re

p_data = [re.sub(r'\n\[[\x20-\x7f]+\]', "", row[2]) for row in data]

In [ ]:
print(data[0][2])

In [ ]:
print(p_data[0])